In [1]:
import pandas as pd 

In [2]:
! mkdir 05_Load

A subdirectory or file 05_Load already exists.


In [3]:
OUTPUT_DIR = "05_Load"

# Read Transform Data

* `representitives_df` will contain the representivies Members of Parliament sourced from https://theyvoteforyou.org.au
* `aec_localities_df` will contain the mapping of voting localities to postcodes and suburbs sourced from https://electorate.aec.gov.au/

In [4]:
representitives_df = pd.read_csv("02_transform_they_vote_for_you/output.csv", dtype="str")
representitives_df.head()

,id,name_first,name_last,electorate,party
0,6,Anthony,Albanese,Grayndler,Australian Labor Party
1,13,Kevin,Andrews,Menzies,Liberal Party
2,40,Sharon,Bird,Cunningham,Australian Labor Party
3,69,Tony,Burke,Watson,Australian Labor Party
4,74,Anthony,Byrne,Holt,Australian Labor Party


In [5]:
aec_localities_df = pd.read_csv("04_transform_aec_electorates/output.csv", dtype="str")
aec_localities_df.head()

,electorate,locality_suburb,postcode,state
0,Adelaide,Adelaide,5000,SA
1,Adelaide,Allenby Gardens,5009,SA
2,Adelaide,Angle Park,5010,SA
3,Adelaide,Ashford,5035,SA
4,Adelaide,Blair Athol,5084,SA


# Normalising Dataframes

There are a number of columns in the dataframes that could be modelled as separate tables

* `aec_localities_df["state"]`
* `aec_localities_df["locality_suburb"]`
* `aec_localities_df["electorate"]`
* `representitives_df["electorate"]`
* `representitives_df["party"]`

In [6]:
def convert_column_to_fk(df, column, output_dir):
    
    values = sorted(list(df[column].unique()))
    ids = range(1, len(values) + 1)    
    
    fk_df = pd.DataFrame(values, index = ids).reset_index()
    fk_df.columns = ["id", column]
    fk_df.to_csv(f"{output_dir}/{column}.csv", index = False)
    
    df[column] = df[column].replace(dict(zip(values, ids)))
    df = df.rename(columns = {column: f"{column}_id"})

    return df

# Electorates

Extract electorates to it's own csv, then ensure that the extracted electorates exist in both my `representivies_df` and `aec_localitities_df`.

In [7]:
sorted(representitives_df["electorate"].unique()) == sorted(aec_localities_df["electorate"].unique())

True

In [8]:
aec_localities_df = convert_column_to_fk(aec_localities_df, "electorate", OUTPUT_DIR)
aec_localities_df.head()

,electorate_id,locality_suburb,postcode,state
0,1,Adelaide,5000,SA
1,1,Allenby Gardens,5009,SA
2,1,Angle Park,5010,SA
3,1,Ashford,5035,SA
4,1,Blair Athol,5084,SA


In [9]:
electorates_df = pd.read_csv(f"{OUTPUT_DIR}/electorate.csv")
electorates_df.head()

,id,electorate
0,1,Adelaide
1,2,Aston
2,3,Ballarat
3,4,Banks
4,5,Barker


In [10]:
representitives_df.head()

,id,name_first,name_last,electorate,party
0,6,Anthony,Albanese,Grayndler,Australian Labor Party
1,13,Kevin,Andrews,Menzies,Liberal Party
2,40,Sharon,Bird,Cunningham,Australian Labor Party
3,69,Tony,Burke,Watson,Australian Labor Party
4,74,Anthony,Byrne,Holt,Australian Labor Party


In [11]:
electorate_replacments = dict(zip(electorates_df["electorate"].values, electorates_df["id"].values))

In [12]:
representitives_df["electorate"] = representitives_df["electorate"].replace(electorate_replacments)
representitives_df = representitives_df.rename(columns={"electorate": "electorate_id"})
representitives_df.head()

,id,name_first,name_last,electorate_id,party
0,6,Anthony,Albanese,65,Australian Labor Party
1,13,Kevin,Andrews,109,Liberal Party
2,40,Sharon,Bird,38,Australian Labor Party
3,69,Tony,Burke,145,Australian Labor Party
4,74,Anthony,Byrne,75,Australian Labor Party


# Normalise Representitives

In [13]:
representitives_df.dtypes

id               object
name_first       object
name_last        object
electorate_id     int64
party            object
dtype: object

In [14]:
representitives_df = convert_column_to_fk(representitives_df, "party", OUTPUT_DIR)
representitives_df.head()

,id,name_first,name_last,electorate_id,party_id
0,6,Anthony,Albanese,65,2
1,13,Kevin,Andrews,109,8
2,40,Sharon,Bird,38,2
3,69,Tony,Burke,145,2
4,74,Anthony,Byrne,75,2


In [15]:
representitives_df.dtypes

id               object
name_first       object
name_last        object
electorate_id     int64
party_id          int64
dtype: object

# Normalise AEC Electorates

In [16]:
aec_localities_df.head()

,electorate_id,locality_suburb,postcode,state
0,1,Adelaide,5000,SA
1,1,Allenby Gardens,5009,SA
2,1,Angle Park,5010,SA
3,1,Ashford,5035,SA
4,1,Blair Athol,5084,SA


In [17]:
for col in aec_localities_df.columns:
    print(col, aec_localities_df[col].is_unique)

electorate_id False
locality_suburb False
postcode False
state False


In [18]:
aec_localities_df["state"].unique()

array(['SA', 'VIC', 'NSW', 'TAS', 'ACT', 'QLD', 'WA', 'NT'], dtype=object)

In [19]:
aec_localities_df = convert_column_to_fk(aec_localities_df, "state", OUTPUT_DIR)
aec_localities_df.head()

,electorate_id,locality_suburb,postcode,state_id
0,1,Adelaide,5000,5
1,1,Allenby Gardens,5009,5
2,1,Angle Park,5010,5
3,1,Ashford,5035,5
4,1,Blair Athol,5084,5


In [20]:
aec_localities_df["postcode"].describe()

count     17668
unique     2658
top        0872
freq        474
Name: postcode, dtype: object

In [21]:
aec_localities_df = convert_column_to_fk(aec_localities_df, "postcode", OUTPUT_DIR)
aec_localities_df.head()

,electorate_id,locality_suburb,postcode_id,state_id
0,1,Adelaide,1813,5
1,1,Allenby Gardens,1817,5
2,1,Angle Park,1818,5
3,1,Ashford,1838,5
4,1,Blair Athol,1873,5


In [22]:
aec_localities_df = convert_column_to_fk(aec_localities_df, "locality_suburb", OUTPUT_DIR)
aec_localities_df.head()

,electorate_id,locality_suburb_id,postcode_id,state_id
0,1,51,1813,5
1,1,165,1817,5
2,1,279,1818,5
3,1,453,1838,5
4,1,1364,1873,5


# Ensure No Duplicates

In [23]:
aec_localities_df.shape

(17668, 4)

In [24]:
aec_localities_df.drop_duplicates().shape

(17668, 4)

In [25]:
representitives_df.shape

(151, 5)

In [26]:
representitives_df.drop_duplicates().shape

(151, 5)

# Save For Import

In [27]:
representitives_df.to_csv(f"{OUTPUT_DIR}/representitive.csv", index = False)
aec_localities_df.to_csv(f"{OUTPUT_DIR}/aec_locality.csv", index = False)

# Do Import

In [28]:
from sqlalchemy import create_engine
from config.key import postgres_password

This assumes you've already created a database, if not see `resources/create_database.sql`.

In [29]:
connection_string = f"postgres:{postgres_password}@localhost:5432/auspol"
engine = create_engine(f'postgresql://{connection_string}') 

In [30]:
def execute_sql_file(filepath, engine):
    with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as con:
        with open(filepath, "r") as schema_file: 
            return con.execute(schema_file.read(), autocommit=True)    

In [31]:
execute_sql_file("resources/drop_tables.sql", engine)

In [32]:
execute_sql_file("resources/schema.sql", engine)

In [33]:
import_tables = [
    ("state.csv", "states"),
    ("party.csv", "parties"),    
    ("electorate.csv", "electorates"),
    ("postcode.csv", "postcodes"),
    ("locality_suburb.csv", "locality_suburbs"),
    ("representitive.csv", "representitives"),
    ("aec_locality.csv", "localities")
]

for item in import_tables:
    # loading as string as we don't want pandas 
    # to make assumptions about the datatypes
    # i.e. assuming that postcode '0820' and '820'
    # are the same because pandas has read postcode as integers
    print(f"Importing: {item[0]} into table {item[1]}")
    import_df = pd.read_csv(f"{OUTPUT_DIR}/{item[0]}", dtype="str")
    import_df.to_sql(name=item[1], con = engine, index = False, if_exists = "append")    
    print("\tdone!\n")

Importing: state.csv into table states
	done!

Importing: party.csv into table parties
	done!

Importing: electorate.csv into table electorates
	done!

Importing: postcode.csv into table postcodes
	done!

Importing: locality_suburb.csv into table locality_suburbs
	done!

Importing: representitive.csv into table representitives
	done!

Importing: aec_locality.csv into table localities
	done!

